In [2]:
import pandas as pd
import os
import json
from datetime import datetime, timedelta

In [ ]:
excel_path = "data.xlsx"

output_dir = "json_output"
os.makedirs(output_dir, exist_ok=True)
excel_data = pd.read_excel(excel_path, sheet_name=None)

for sheet_name, df in excel_data.items():
    safe_name = sheet_name.replace(" ", "_").replace("/", "_")
    json_file_path = os.path.join(output_dir, f"{safe_name}.json")
    
    df.to_json(json_file_path, orient='records', indent=2)

    print(f"Saved {sheet_name} -> {json_file_path}")

Saved New_Account -> json_output\New_Account.json
Saved New_Card -> json_output\New_Card.json
Saved District -> json_output\District.json
Saved Loan -> json_output\Loan.json
Saved New_Client -> json_output\New_Client.json
Saved New_Disposition -> json_output\New_Disposition.json
Saved New_Transaction -> json_output\New_Transaction.json
Saved Order -> json_output\Order.json


In [10]:
df = pd.read_excel('Data_Dictionary.xlsx')
df.columns = df.columns.str.strip()

df.to_json("dict.json", orient='records', indent=2)

Prepare a dashboard for the All the Accounts doing Credit transactions from Moravia and
Prague. Prepare the Aggregated View of the transactions basis on 2 things i.e.
a. Account Wise
b. Month Wise
c. Account and Month wise

In [ ]:
district_df = pd.read_json('json_output/District.json')
account_df = pd.read_json('json_output/New_Account.json')
transaction_df = pd.read_json('json_output/New_Transaction.json')

target_regions = ['Moravia', 'Prague']
filtered_districts = district_df[district_df['A3'].isin(target_regions)]
target_district_ids = filtered_districts['A1'].tolist()
filtered_accounts = account_df[account_df['district_id'].isin(target_district_ids)]
target_account_ids = filtered_accounts['account_id'].tolist()
credit_txn = transaction_df[
    (transaction_df['account_id'].isin(target_account_ids)) &
    (transaction_df['type'] == 'CREDIT')
].copy()

def parse_ymd(ymd):
    ymd = int(ymd)
    year = 1900 + int(str(ymd)[:2])
    month = int(str(ymd)[2:4])
    return f"{year:04d}-{month:02d}"

credit_txn['month'] = credit_txn['date'].apply(parse_ymd)
agg_account = credit_txn.groupby('account_id')['amount'].sum().reset_index()
agg_account.rename(columns={'amount': 'total_credit'}, inplace=True)
agg_month = credit_txn.groupby('month')['amount'].sum().reset_index()
agg_month.rename(columns={'amount': 'total_credit'}, inplace=True)
agg_account_month = credit_txn.groupby(['account_id', 'month'])['amount'].sum().reset_index()
agg_account_month.rename(columns={'amount': 'total_credit'}, inplace=True)
output = {
    "account_wise": agg_account.to_dict(orient='records'),
    "month_wise": agg_month.to_dict(orient='records'),
    "account_month_wise": agg_account_month.to_dict(orient='records')
}
with open('aggregated_credit_transactions.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Aggregated credit transaction data saved to 'aggregated_credit_transactions.json'")

Aggregated credit transaction data saved to 'aggregated_credit_transactions.json'


Analysis on Highly populated versus Low populated districts. Find out the amount of Credit
and Debit transaction from 5 highly populated and 5 lowest populated areas respectively,
above analysis should be from last 3 months

In [ ]:
top_5 = district_df.nlargest(5, 'A4')
bottom_5 = district_df.nsmallest(5, 'A4')
top_ids = top_5['A1'].tolist()
bottom_ids = bottom_5['A1'].tolist()
top_acc_ids = account_df[account_df['district_id'].isin(top_ids)]['account_id']
bottom_acc_ids = account_df[account_df['district_id'].isin(bottom_ids)]['account_id']

def parse_date(ymd):
    ymd = int(ymd)
    year = 1900 + int(str(ymd)[:2])
    month = int(str(ymd)[2:4])
    day = int(str(ymd)[4:6])
    return datetime(year, month, day)
transaction_df['parsed_date'] = transaction_df['date'].apply(parse_date)
max_date = transaction_df['parsed_date'].max()
cutoff_date = max_date - timedelta(days=90)
txn_recent = transaction_df[
    (transaction_df['parsed_date'] >= cutoff_date) &
    (transaction_df['type'].isin(['CREDIT', 'DEBIT']))
]
def summarize_group(acc_ids, label):
    group_txn = txn_recent[txn_recent['account_id'].isin(acc_ids)]
    summary = group_txn.groupby('type')['amount'].sum().reset_index()
    summary['group'] = label
    return summary
top_summary = summarize_group(top_acc_ids, 'Top 5 Populated')
bottom_summary = summarize_group(bottom_acc_ids, 'Bottom 5 Populated')
final_summary = pd.concat([top_summary, bottom_summary], ignore_index=True)
final_summary.to_json('populated_district_txn_summary.json', orient='records', indent=2)

print(final_summary)

     type      amount               group
0  CREDIT  58737742.9     Top 5 Populated
1   DEBIT  53807327.0     Top 5 Populated
2  CREDIT  13801016.8  Bottom 5 Populated
3   DEBIT  12926596.8  Bottom 5 Populated


How many cards are issued to mid age females?

In [ ]:
clients = pd.read_json("json_output/New_Client.json")
disps = pd.read_json("json_output/New_Disposition.json")
cards = pd.read_json("json_output/New_Card.json")

mid_age_females = clients[
    (clients["gender"] == "FEMALE") & 
    (clients["age_levels"].str.upper() == "MIDDLE AGED")
]
mid_female_ids = mid_age_females["client_id"]
relevant_disps = disps[disps["client_id"].isin(mid_female_ids)]
issued_cards = cards[cards["disp_id"].isin(relevant_disps["disp_id"])]
num_cards = len(issued_cards)
print(f"Cards issued to mid-age females: {num_cards}")


Cards issued to mid-age females: 227


Number of cards issued in district where average salary is more than 9000, is it a good
strategy?

In [45]:
district_df = pd.read_json('json_output/District.json')
account_df = pd.read_json('json_output/New_Account.json')
disposition_df = pd.read_json('json_output/New_Disposition.json')
card_df = pd.read_json('json_output/New_Card.json')

rich_district_ids = district_df[district_df['A11'] > 9000]['A1']
rich_accounts = account_df[account_df['district_id'].isin(rich_district_ids)]['account_id']
rich_disps = disposition_df[disposition_df['account_id'].isin(rich_accounts)]
cards_in_rich_districts = card_df[card_df['disp_id'].isin(rich_disps['disp_id'])]
card_count = len(cards_in_rich_districts)
print(f"Number of cards issued in districts with average salary > 9000: {card_count}")

Number of cards issued in districts with average salary > 9000: 437


Are we providing loans to members belonging to district where committed crimes are more
than 6000 for code 95, if yes then provide the number of loans per district?

In [66]:
district_df = pd.read_json('json_output/District.json')
account_df = pd.read_json('json_output/New_Account.json')
loan_df = pd.read_json('json_output/Loan.json')

district_df['A15'] = pd.to_numeric(district_df['A15'], errors='coerce')
crime_districts = district_df[district_df['A15'] > 6000]
crime_district_ids = crime_districts['A1'].tolist()
accounts_in_crime_districts = account_df[account_df['district_id'].isin(crime_district_ids)]
loans_in_crime_districts = loan_df.merge(
    accounts_in_crime_districts[['account_id', 'district_id']],
    on='account_id'
)
loan_counts = loans_in_crime_districts.groupby('district_id').size().reset_index(name='loan_count')
loan_counts = loan_counts.merge(
    district_df[['A1', 'A2', 'A15']],
    left_on='district_id',
    right_on='A1'
)[['district_id', 'A2', 'A15', 'loan_count']]
print("Loans in districts with >6000 crimes (1995):")
print(loan_counts)
loan_counts.to_json('loans_in_high_crime_districts.json', orient='records', indent=2)

Loans in districts with >6000 crimes (1995):
   district_id                A2      A15  loan_count
0            1       Hl.m. Praha  85677.0          84
1           14  Ceske Budejovice   6604.0           8
2           26     Plzen - mesto   6041.0           6
3           40           Teplice   6949.0           6
4           41    Usti nad Labem   6445.0           3
5           47         Pardubice   6079.0          10
6           54      Brno - mesto  18721.0          24
7           70           Karvina   9878.0          24
8           72           Olomouc   9672.0          14
9           74   Ostrava - mesto  18782.0          20


Create profile of customers in accordance of districts where max money is being paid to
a. Insurance.
b. Household
c. Leasing
d. Loan

In [48]:
transactions = pd.read_json("json_output/New_Transaction.json")
accounts = pd.read_json("json_output/New_Account.json")
dispositions = pd.read_json("json_output/New_Disposition.json")
clients = pd.read_json("json_output/New_Client.json")
districts = pd.read_json("json_output/District.json")

txn_acc = transactions.merge(accounts[['account_id', 'district_id']], on='account_id', how='left')
categories = {
    "Insurance": "insurance",
    "Household": "household",
    "Leasing": "leasing",
    "Loan": "loan"
}

profiles = {}
for label, symbol in categories.items():
    cat_txn = txn_acc[txn_acc['k_symbol'].str.lower().fillna('').str.contains(symbol)]
    district_sum = cat_txn.groupby('district_id')['amount'].sum().reset_index()

    if district_sum.empty:
        continue
    top_row = district_sum.loc[district_sum['amount'].idxmax()]
    top_district_id = top_row['district_id']
    total_amount = top_row['amount']
    district_name = districts[districts['A1'] == top_district_id]['A2'].values[0]
    top_accounts = accounts[accounts['district_id'] == top_district_id]['account_id']
    disp = dispositions[dispositions['account_id'].isin(top_accounts)]
    client_profiles = clients[clients['client_id'].isin(disp['client_id'])]
    profile_summary = client_profiles[['client_id', 'gender', 'age', 'age_levels']].to_dict(orient='records')
    profiles[label] = {
        "district_id": int(top_district_id),
        "district_name": district_name,
        "total_amount_paid": float(total_amount),
        "num_customers": len(profile_summary),
        "customers": profile_summary
    }
with open("category_wise_customer_profiles.json", "w") as f:
    json.dump(profiles, f, indent=2)


Create profile of customers in accordance of districts for the status of loan payment, there will
be 4 categories.
Relate the output of above with district conditions like Crime, Unemployment Rate and
Average Salary.

In [50]:
loan_df = pd.read_json("json_output/Loan.json")
account_df = pd.read_json("json_output/New_Account.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
client_df = pd.read_json("json_output/New_Client.json")
district_df = pd.read_json("json_output/District.json")

loan_df = loan_df.merge(account_df[['account_id', 'district_id']], on='account_id', how='left')
loan_df = loan_df.merge(disp_df[['account_id', 'client_id']], on='account_id', how='left')
loan_df = loan_df.merge(client_df[['client_id', 'gender', 'age', 'age_levels']], on='client_id', how='left')
loan_df = loan_df.merge(district_df, left_on='district_id', right_on='A1', how='left')
profile_by_status = {}
for status in ['A', 'B', 'C', 'D']:
    df_status = loan_df[loan_df['status'] == status]
    customer_info = df_status[[
        'client_id', 'gender', 'age', 'age_levels',
        'district_id', 'A2', 'A11', 'A12', 'A13', 'A15'
    ]].drop_duplicates()
    customer_info.rename(columns={
        'A2': 'district_name',
        'A11': 'average_salary',
        'A12': 'unemployment_95',
        'A13': 'unemployment_96',
        'A15': 'crimes_95'
    }, inplace=True)
    profile_by_status[status] = {
        "status_meaning": {
            "A": "Finished - No problems",
            "B": "Finished - Loan not paid",
            "C": "Running - OK",
            "D": "Running - In debt"
        }[status],
        "num_customers": len(customer_info),
        "customers": customer_info.to_dict(orient='records')
    }
with open("loan_status_customer_profiles.json", "w") as f:
    json.dump(profile_by_status, f, indent=2)

Owners from which district are issuing permanent orders and asking for a loan

In [ ]:
loan_df = pd.read_json("json_output/Loan.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
account_df = pd.read_json("json_output/New_Account.json")
district_df = pd.read_json("json_output/District.json")

owners_df = disp_df[disp_df["type"].str.upper() == "OWNER"]
owners_with_loans = owners_df.merge(loan_df, on="account_id", how="inner")
owners_with_loans = owners_with_loans.merge(account_df[["account_id", "district_id"]], on="account_id", how="left")
owners_with_loans = owners_with_loans.merge(district_df[["A1", "A2"]], left_on="district_id", right_on="A1", how="left")
district_counts = owners_with_loans["A2"].value_counts().reset_index()
district_counts.columns = ["district_name", "num_owners_with_loans"]
print(district_counts)
district_counts.to_json('loans_in_high_crime_districts.json', orient='records', indent=2)

        district_name  num_owners_with_loans
0         Hl.m. Praha                     84
1             Karvina                     24
2        Brno - mesto                     24
3     Ostrava - mesto                     20
4                Zlin                     17
..                ...                    ...
72              Pisek                      3
73             Semily                      3
74            Sokolov                      2
75          Domazlice                      2
76  Jablonec n. Nisou                      1

[77 rows x 2 columns]


Can we say customers from Bohemia are the ones having more male customers possessing
Gold cards in comparison of Moravia?

In [52]:
card_df = pd.read_json("json_output/New_Card.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
client_df = pd.read_json("json_output/New_Client.json")
district_df = pd.read_json("json_output/District.json")

gold_cards = card_df[card_df["type"].str.upper() == "GOLD"]
gold_disp = gold_cards.merge(disp_df, on="disp_id", how="left")
gold_clients = gold_disp.merge(client_df[["client_id", "gender", "district_id"]], on="client_id", how="left")
gold_clients = gold_clients.merge(district_df[["A1", "A2", "A3"]], left_on="district_id", right_on="A1", how="left")
male_gold_clients = gold_clients[gold_clients["gender"].str.upper() == "MALE"]
region_counts = male_gold_clients["A3"].value_counts().reset_index()
region_counts.columns = ["region", "num_male_gold_customers"]

print(region_counts)

            region  num_male_gold_customers
0    north Moravia                       11
1           Prague                        7
2    north Bohemia                        7
3    south Moravia                        7
4  central Bohemia                        7
5     west Bohemia                        6
6     east Bohemia                        5
7    south Bohemia                        3


How many customers having credit card are also availing the loan facilities.


In [ ]:
card_df = pd.read_json("json_output/New_Card.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
loan_df = pd.read_json("json_output/Loan.json")

card_disp = card_df.merge(disp_df[['disp_id', 'client_id']], on='disp_id', how='left')
clients_with_cards = set(card_disp['client_id'].dropna().unique())
loan_disp = loan_df.merge(disp_df[['account_id', 'client_id']], on='account_id', how='left')
clients_with_loans = set(loan_disp['client_id'].dropna().unique())
common_clients = clients_with_cards.intersection(clients_with_loans)
print(f"Customers with both a Credit Card and Loan: {len(common_clients)}")

Customers with both a Credit Card and Loan: 170


Can we say that customers having Classic and Junior card are the ones who are more in debt.

In [ ]:
card_df = pd.read_json("json_output/New_Card.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
loan_df = pd.read_json("json_output/Loan.json")

card_df["type"] = card_df["type"].str.upper()
card_holders = card_df.merge(disp_df[["disp_id", "client_id", "account_id"]], on="disp_id", how="left")
total_holders = card_holders["type"].value_counts().reset_index()
total_holders.columns = ["card_type", "total_holders"]
loan_debt = loan_df[loan_df["status"] == "D"]
loan_clients = loan_debt.merge(disp_df[["account_id", "client_id"]], on="account_id", how="left")
debt_card_holders = card_holders.merge(loan_clients, on="client_id", how="inner")
debt_count = debt_card_holders["type"].value_counts().reset_index()
debt_count.columns = ["card_type", "num_in_debt"]
merged = total_holders.merge(debt_count, on="card_type", how="left").fillna(0)
merged["debt_rate (%)"] = round((merged["num_in_debt"] / merged["total_holders"]) * 100, 2)
merged.to_json("card_debt_rate.json", orient="records", indent=2)

How will you analyze the performance of Mid age vs adults in terms of loan repayments.

In [65]:
loan_df = pd.read_json("json_output/Loan.json")
disp_df = pd.read_json("json_output/New_Disposition.json")
client_df = pd.read_json("json_output/New_Client.json")

loan_disp = loan_df.merge(disp_df[["account_id", "client_id"]], on="account_id", how="left")
loan_with_age = loan_disp.merge(client_df[["client_id", "age_levels"]], on="client_id", how="left")
loan_with_age = loan_with_age[loan_with_age["age_levels"].isin(["MIDDLE AGED", "ADULT"])]
summary = loan_with_age.groupby(["age_levels", "status"]).size().unstack(fill_value=0)
summary_percent = summary.div(summary.sum(axis=1), axis=0).round(2) * 100

print("Absolute Counts:\n", summary)
print("\nPercentages:\n", summary_percent)

summary_json = {
    "absolute": summary.reset_index().to_dict(orient="records"),
    "percentages": summary_percent.reset_index().to_dict(orient="records")
}

with open("loan_repayment_by_age.json", "w") as f:
    json.dump(summary_json, f, indent=2)

Absolute Counts:
 status         A   B    C   D
age_levels                   
ADULT         59   4  129  13
MIDDLE AGED  162  22  303  25

Percentages:
 status          A    B     C    D
age_levels                       
ADULT        29.0  2.0  63.0  6.0
MIDDLE AGED  32.0  4.0  59.0  5.0
